In [178]:
import numpy as np
import pandas as pd
from numpy import nan
import math

In [179]:
df = pd.read_csv('data/df_periods.csv')

In [180]:
# create a datetime column for inspection date
df['inspect_date'] = pd.to_datetime(df.inspection_date)

In [181]:
df.head(2)

,Unnamed: 0,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,...,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,short_inspect_date,short_violation_id,inspect_date
0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",...,1757_20170928,9/28/2017 0:00,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk,20170928,103131,2017-09-28
1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",...,4864_20161206,12/6/2016 0:00,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk,20161206,103157,2016-12-06


In [182]:
# df_short is full rows with 4 columns only
df_short = df[['business_id', 'inspect_date', 'risk_category', 'short_violation_id']]

In [183]:
a = df_short['short_violation_id']
a.isnull().any()

False

In [184]:
df_short['short_violation_id'] = df_short['short_violation_id'].apply(np.int64)

/home/soong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [185]:
df_short.iloc[29]

business_id                            61
inspect_date          2015-08-27 00:00:00
risk_category                    Low Risk
short_violation_id                 103149
Name: 29, dtype: object

In [186]:
len(df_short['business_id'].unique())

5229

In [187]:
df_short.business_id.value_counts()

78328    42
489      42
72049    39
63082    38
79961    36
3895     35
63763    34
79248    34
74283    34
58673    34
60354    33
70996    33
76960    32
4614     32
1641     32
69016    32
2357     31
66874    31
6729     30
69205    30
66961    29
83351    29
71012    29
33920    29
69884    29
85697    29
69799    28
2659     28
4607     28
59828    28
         ..
59154     1
89772     1
86652     1
8882      1
91986     1
78928     1
3899      1
90042     1
90914     1
79783     1
83969     1
11488     1
69574     1
85680     1
93448     1
91042     1
86960     1
5980      1
69664     1
6021      1
2971      1
86110     1
34647     1
91170     1
89900     1
39025     1
85063     1
2261      1
83571     1
62816     1
Name: business_id, Length: 5229, dtype: int64

### Let's aggregate risk_category and short_violation_id group by business_id and short_inspection_date

In [188]:
# Let's use df_short to do the aggregation

In [189]:
n305 = df_short.groupby(['business_id', 'inspect_date'])['risk_category'].unique()

In [190]:
type(n305)

pandas.core.series.Series

In [191]:
list_n305 = n305.tolist()

In [192]:
m259 = df_short.groupby(['business_id', 'inspect_date'])['short_violation_id'].unique()

In [193]:
df_grouped = pd.DataFrame(m259)

In [194]:
df_grouped.head(3)

short_violation_id
business_id inspect_date                          
19          2016-05-13    [103144, 103157, 103154]
            2017-12-11            [103144, 103116]
24          2016-03-11                    [103154]

In [195]:
#df_grouped.loc[df_grouped['business_id'==19]]
bid_vnum = df_grouped.reset_index().groupby('business_id')['inspect_date'].nunique()
type(bid_vnum)

pandas.core.series.Series

In [196]:
bid_vnum.sort_values(axis=0, ascending=False)

business_id
63082    8
72049    7
86643    7
3895     7
5875     7
5905     7
5854     7
2692     7
5908     7
5634     7
76233    6
5925     6
69884    6
80887    6
1381     6
69799    6
2418     6
4686     6
2420     6
5805     6
64161    6
580      6
33413    6
79248    6
5950     6
82176    6
5797     6
69909    6
79525    6
71154    6
        ..
5831     1
80299    1
80279    1
5839     1
80253    1
5789     1
5786     1
5509     1
81033    1
81167    1
5524     1
5528     1
5533     1
81140    1
81104    1
5605     1
5606     1
81064    1
80974    1
5744     1
80961    1
80935    1
5630     1
80861    1
80820    1
80819    1
80800    1
5648     1
5652     1
94944    1
Name: inspect_date, Length: 5229, dtype: int64

## Let's look at restaurants with more than 5 violation dates

In [197]:
len(bid_vnum[bid_vnum.values >5])

74

In [198]:
# List the restaurants with more than 5 violation dates
more_than5 = bid_vnum[bid_vnum.values >5]

In [199]:
# business ids with more than 5 violation dates
bid_mt5 = more_than5.index
bid_mt5
bid_mt5=list(bid_mt5)

In [200]:
# Show bid_mt5 restaurants from df_grouped
df_grouped.loc[bid_mt5]

short_violation_id
business_id inspect_date                                                   
489         2015-07-28    [103154, 103144, 103147, 103124, 103131, 10316...
            2015-08-07                             [103144, 103147, 103142]
            2016-03-08     [103102, 103149, 103133, 103114, 103103, 103134]
            2016-07-21    [103130, 103144, 103103, 103156, 103149, 10316...
            2016-12-20    [103124, 103150, 103154, 103131, 103149, 10312...
            2017-11-30    [103133, 103120, 103150, 103149, 103109, 10314...
580         2015-08-20                                             [103105]
            2016-03-11                                     [103144, 103105]
            2016-08-29                             [103154, 103149, 103119]
            2017-03-07                                     [103154, 103142]
            2017-08-29                                             [103153]
            2018-03-13                                     [103154, 103120]
792         2016-04-14                                     [103120, 103145]
            2016-10-17                             [103132, 103150, 103120]
            2017-07-13                             [103131, 103124, 103103]
            2017-07-24                                             [103120]
            2017-08-07                                             [103103]
            2018-01-23                                     [103161, 103154]
1261        2015-06-10                             [103157, 103131, 103119]
            2015-06-11                                     [103114, 103157]
            2015-12-15                             [103157, 103119, 103131]
            2015-12-29                                     [103114, 103116]
            2017-12-01             [103156, 103131, 103154, 103157, 103139]
            2017-12-12                                             [103139]
1381        2016-03-30     [103144, 103110, 103114, 103103, 103149, 103156]
            2016-04-15                                     [103161, 103120]
            2016-11-28                     [103119, 103157, 103139, 103103]
            2017-08-22                                     [103120, 103116]
            2017-09-14                                     [103133, 103120]
            2017-09-20                                             [103154]
...                                                                     ...
82176       2016-09-12                                     [103149, 103157]
            2017-07-14                                     [103129, 103116]
            2017-07-28                                     [103129, 103116]
            2017-08-15                                     [103129, 103116]
            2018-01-05                                             [103157]
82345       2015-06-15                                     [103148, 103162]
            2016-06-24     [103157, 103119, 103154, 103106, 103144, 103156]
            2017-02-14     [103124, 103156, 103120, 103136, 103154, 103144]
            2017-02-23                                     [103144, 103124]
            2017-10-12             [103156, 103149, 103144, 103103, 103154]
            2017-10-17                                             [103144]
85697       2016-02-18                     [103124, 103142, 103136, 103154]
            2016-12-02    [103156, 103154, 103124, 103157, 103133, 10314...
            2016-12-22                     [103144, 103154, 103119, 103118]
            2017-06-22     [103119, 103156, 103154, 103144, 103120, 103149]
            2017-07-07     [103154, 103144, 103136, 103120, 103135, 103116]
            2017-07-14                                     [103154, 103135]
86301       2016-02-10                                             [103139]
            2016-03-24                                             [103120]
            2016-09-23                             [103149, 103119, 103139]
            2017-03-17       

## Let's look at restaurants with 5 violation dates

In [201]:
len(bid_vnum[bid_vnum.values ==5])

147

In [202]:
# List the restaurants with 5 violation dates
with5 = bid_vnum[bid_vnum.values == 5]

In [203]:
# business ids with 5 violation dates
bid_with5 = with5.index
bid_with5
bid_with5=list(bid_with5)

In [204]:
# Show bid_mt4 restaurants from df_grouped
df_grouped.loc[bid_with5]

short_violation_id
business_id inspect_date                                                   
67          2015-05-20                             [103119, 103124, 103145]
            2016-04-01                     [103154, 103144, 103161, 103109]
            2016-10-19                                     [103154, 103124]
            2017-08-04     [103150, 103120, 103142, 103114, 103147, 103139]
            2018-02-20                     [103131, 103154, 103120, 103149]
217         2015-08-18                             [103154, 103149, 103145]
            2016-03-29                             [103144, 103154, 103142]
            2016-09-12                     [103120, 103149, 103154, 103145]
            2017-03-30                             [103142, 103131, 103144]
            2017-09-25                                             [103103]
522         2015-06-30                                             [103134]
            2015-11-24                                             [103103]
            2016-07-06                                     [103124, 103156]
            2017-02-09                     [103131, 103154, 103124, 103156]
            2017-09-22                             [103119, 103156, 103154]
619         2015-04-23                                             [103103]
            2016-05-25             [103149, 103142, 103154, 103144, 103120]
            2016-06-14                                             [103147]
            2017-01-18             [103139, 103121, 103144, 103154, 103118]
            2017-09-29                     [103149, 103144, 103136, 103121]
1144        2015-12-22             [103154, 103114, 103109, 103108, 103149]
            2016-03-29             [103131, 103138, 103144, 103120, 103154]
            2016-09-29     [103124, 103103, 103154, 103131, 103156, 103139]
            2017-08-28                     [103131, 103145, 103144, 103133]
            2018-02-26             [103147, 103139, 103154, 103142, 103131]
1434        2015-07-09                                     [103120, 103161]
            2016-03-07                                     [103103, 103149]
            2016-03-14                                             [103120]
            2016-12-05                                     [103141, 103149]
            2017-12-01                             [103161, 103145, 103124]
...                                                                     ...
81606       2016-02-22                     [103145, 103154, 103129, 103116]
            2016-09-22             [103139, 103149, 103144, 103109, 103120]
            2016-10-04                             [103123, 103149, 103144]
            2017-05-31             [103149, 103131, 103144, 103157, 103154]
            2017-06-15                                             [103144]
82134       2015-04-27    [103133, 103142, 103105, 103102, 103103, 10310...
            2015-04-28                                             [103131]
            2015-12-07             [103102, 103144, 103116, 103132, 103138]
            2015-12-21                                     [103124, 103120]
            2017-11-28    [103144, 103149, 103142, 103157, 103132, 10313...
84517       2015-08-10                     [103148, 103144, 103154, 103157]
            2015-12-04                                     [103139, 103149]
            2016-07-27                             [103120, 103154, 103144]
            2017-04-07                             [103149, 103120, 103145]
            2017-12-13                                     [103144, 103151]
85166       2015-10-30             [103152, 103145, 103149, 103161, 103144]
            2016-10-05                     [103145, 103161, 103149, 103152]
            2017-08-23                     [103124, 103161, 103133, 103103]
            2017-11-20                                     [103120, 103139]
            2018-02-06    [103149, 103103, 103129, 103139, 103133, 10311...
86658       2016-03-10    [10

## Let's look at restaurants with 4 violation dates

In [205]:
len(bid_vnum[bid_vnum.values ==4])

508

In [206]:
with4 = bid_vnum[bid_vnum.values == 4]
bid_with4 = with4.index
bid_with4
bid_with4=list(bid_with4)
df_grouped.loc[bid_with4]

short_violation_id
business_id inspect_date                                                   
45          2016-01-04    [103144, 103154, 103118, 103116, 103119, 10314...
            2016-06-14             [103154, 103144, 103131, 103133, 103116]
            2017-03-07             [103149, 103157, 103131, 103142, 103154]
            2017-09-14                     [103120, 103142, 103154, 103114]
56          2015-12-22                             [103133, 103124, 103120]
            2016-08-02                     [103144, 103154, 103119, 103149]
            2017-04-20                             [103129, 103149, 103156]
            2017-05-18                                             [103129]
61          2015-08-27                             [103133, 103142, 103149]
            2016-07-08                             [103142, 103139, 103150]
            2017-01-24                                             [103153]
            2017-11-28                                     [103133, 103144]
95          2015-12-10                     [103131, 103124, 103154, 103142]
            2016-07-18             [103144, 103142, 103124, 103154, 103131]
            2017-03-07                     [103154, 103142, 103143, 103149]
            2017-10-16                     [103154, 103124, 103116, 103114]
180         2016-01-06                     [103119, 103120, 103157, 103149]
            2016-06-17                                             [103149]
            2016-12-20                                     [103149, 103119]
            2017-08-15                                     [103119, 103149]
273         2015-05-19    [103145, 103149, 103119, 103124, 103133, 10314...
            2015-05-26                                             [103142]
            2016-09-20    [103149, 103139, 103133, 103150, 103157, 10312...
            2017-10-02    [103157, 103120, 103114, 103142, 103149, 10314...
279         2015-10-20                                     [103131, 103142]
            2016-05-18                                             [103161]
            2017-01-04                             [103132, 103161, 103120]
            2017-10-10                             [103144, 103133, 103147]
298         2015-08-28                     [103119, 103114, 103109, 103154]
            2016-04-08                             [103120, 103102, 103154]
...                                                                     ...
86910       2017-08-21                                     [103133, 103131]
            2018-03-14                                             [103120]
87613       2016-06-20                     [103105, 103124, 103120, 103114]
            2017-03-06             [103161, 103133, 103105, 103149, 103139]
            2017-05-16                                             [103131]
            2017-09-08                     [103154, 103120, 103107, 103147]
87699       2016-07-07                             [103139, 103149, 103144]
            2016-12-22                                             [103154]
            2017-07-25                                             [103131]
            2018-01-18                                             [103144]
87802       2016-06-07                                     [103103, 103154]
            2016-12-06                                             [103142]
            2017-08-15                                     [103145, 103131]
            2018-03-06                                             [103142]
88326       2016-10-07                                     [103105, 103107]
            2017-02-07                     [103144, 103149, 103105, 103120]
            2017-10-05                             [103149, 103116, 103119]
            2018-02-27                                     [103119, 103124]
88657       2016-08-30                     [103120, 103119, 103124, 103139]
            2016-09-21                                             [103154]
            2017-04-13       

## Let's look at restaurants with 3 violation dates

In [207]:
len(bid_vnum[bid_vnum.values ==3])

1297

In [208]:
with3 = bid_vnum[bid_vnum.values == 3]
bid_with3 = with3.index
bid_with3
bid_with3=list(bid_with3)
df_grouped.loc[bid_with3]

short_violation_id
business_id inspect_date                                                   
24          2016-03-11                                             [103154]
            2016-10-05                                             [103154]
            2017-11-01                                             [103139]
73          2015-07-17             [103161, 103138, 103145, 103139, 103103]
            2016-09-27                             [103149, 103154, 103131]
            2017-11-29                     [103124, 103144, 103150, 103131]
77          2016-03-11                                     [103103, 103144]
            2016-10-05                                     [103154, 103139]
            2017-11-01                                     [103124, 103139]
88          2015-08-24                             [103103, 103156, 103154]
            2017-01-11                             [103142, 103124, 103139]
            2017-10-16                     [103154, 103156, 103107, 103103]
98          2015-06-29                                     [103154, 103131]
            2016-03-16                                             [103116]
            2017-10-03                                     [103154, 103119]
101         2016-05-25                                             [103154]
            2017-06-27                             [103149, 103116, 103129]
            2018-01-29                                     [103129, 103116]
102         2016-11-03                     [103157, 103142, 103109, 103119]
            2017-07-28                                     [103124, 103120]
            2018-03-02                     [103124, 103147, 103133, 103129]
140         2016-04-14                     [103154, 103119, 103156, 103103]
            2017-10-11     [103120, 103139, 103131, 103129, 103143, 103109]
            2018-01-05                     [103156, 103124, 103120, 103131]
146         2015-07-23                                     [103149, 103103]
            2016-04-07             [103161, 103120, 103133, 103119, 103149]
            2016-10-17             [103120, 103124, 103116, 103139, 103154]
155         2015-06-10                             [103157, 103114, 103116]
            2015-12-29                                             [103114]
            2017-02-27             [103120, 103114, 103154, 103124, 103133]
...                                                                     ...
88525       2016-08-01                             [103142, 103154, 103150]
            2017-08-08                             [103142, 103154, 103161]
            2018-02-27                                     [103161, 103142]
88722       2016-11-01                     [103144, 103157, 103145, 103138]
            2017-03-24                             [103139, 103157, 103158]
            2017-12-13                                             [103157]
88799       2016-08-19             [103148, 103139, 103119, 103144, 103124]
            2017-03-06                     [103139, 103120, 103119, 103105]
            2017-09-18             [103114, 103103, 103139, 103154, 103133]
88899       2016-09-09             [103157, 103129, 103120, 103154, 103144]
            2017-02-23                                     [103154, 103144]
            2017-10-11                     [103157, 103156, 103120, 103154]
89072       2016-09-22                             [103149, 103131, 103120]
            2017-02-21                             [103149, 103103, 103114]
            2017-12-01                                     [103154, 103114]
89086       2017-02-28                             [103131, 103103, 103144]
            2017-07-20                             [103161, 103127, 103145]
            2018-01-30                                     [103131, 103124]
89305       2016-09-30                             [103154, 103120, 103157]
            2017-09-27    [103102, 103120, 103156, 103139, 103154, 10315...
            2017-10-12       

## Let's look at restaurants with 2 violation dates

In [209]:
len(bid_vnum[bid_vnum.values ==2])

1839

In [210]:
with2 = bid_vnum[bid_vnum.values == 2]
bid_with2 = with2.index
bid_with2
bid_with2 =list(bid_with2)
df_grouped.loc[bid_with2]

short_violation_id
business_id inspect_date                                                   
19          2016-05-13                             [103144, 103157, 103154]
            2017-12-11                                     [103144, 103116]
58          2016-04-07     [103103, 103109, 103105, 103148, 103147, 103161]
            2017-09-18    [103129, 103105, 103116, 103124, 103139, 10310...
66          2016-03-22                                             [103149]
            2017-07-03                                     [103154, 103143]
80          2016-04-12                                             [103139]
            2017-12-04                                     [103162, 103157]
81          2015-03-30                                             [103120]
            2017-12-05                                             [103162]
99          2015-08-11    [103147, 103145, 103131, 103149, 103139, 10313...
            2017-12-07     [103131, 103155, 103124, 103177, 103133, 103139]
108         2015-04-17                                     [103133, 103144]
            2017-01-06                                     [103157, 103154]
116         2016-05-27     [103157, 103118, 103150, 103105, 103109, 103156]
            2017-05-09                                             [103120]
121         2015-10-20                     [103119, 103138, 103124, 103131]
            2017-04-18                             [103150, 103124, 103127]
125         2016-05-20                             [103161, 103124, 103154]
            2017-07-19                     [103161, 103124, 103150, 103144]
134         2015-08-06                                             [103145]
            2017-02-14                                             [103112]
151         2015-05-20                             [103103, 103138, 103116]
            2016-07-26                                     [103129, 103124]
184         2016-10-03                                             [103149]
            2018-02-13                             [103131, 103157, 103129]
192         2016-06-28                             [103131, 103139, 103120]
            2017-06-26                                     [103109, 103103]
194         2015-04-20                     [103120, 103157, 103108, 103102]
            2017-09-19     [103149, 103138, 103120, 103131, 103154, 103145]
...                                                                     ...
91205       2017-04-28                                             [103160]
            2018-02-15                                     [103105, 103161]
91313       2017-11-03             [103132, 103119, 103133, 103149, 103105]
            2018-03-06                                             [103162]
91334       2017-05-19                             [103144, 103124, 103119]
            2018-01-30                                     [103161, 103154]
91390       2017-05-05     [103124, 103150, 103161, 103102, 103116, 103133]
            2017-12-18                     [103103, 103109, 103154, 103133]
91459       2017-05-11                             [103144, 103133, 103139]
            2017-10-18    [103103, 103142, 103144, 103119, 103133, 10315...
91473       2017-07-20                             [103139, 103154, 103124]
            2018-02-27                     [103105, 103144, 103154, 103109]
91802       2017-08-10                                     [103161, 103138]
            2018-02-28                                     [103139, 103150]
91829       2017-06-01                                     [103154, 103139]
            2017-12-07                                             [103119]
91830       2017-07-22                     [103150, 103162, 103154, 103120]
            2018-02-27     [103156, 103154, 103144, 103120, 103157, 103119]
91853       2017-06-07                                     [103124, 103103]
            2017-11-17                             [103120, 103149, 103154]
91999       2017-07-18       

In [211]:
df_grouped['risk_category'] = list_n305
df_grouped

short_violation_id  \
business_id inspect_date                                                      
19          2016-05-13                             [103144, 103157, 103154]   
            2017-12-11                                     [103144, 103116]   
24          2016-03-11                                             [103154]   
            2016-10-05                                             [103154]   
            2017-11-01                                             [103139]   
31          2015-12-04                                             [103157]   
45          2016-01-04    [103144, 103154, 103118, 103116, 103119, 10314...   
            2016-06-14             [103154, 103144, 103131, 103133, 103116]   
            2017-03-07             [103149, 103157, 103131, 103142, 103154]   
            2017-09-14                     [103120, 103142, 103154, 103114]   
48          2018-02-16                                     [103120, 103154]   
54          2017-02-15                             [103120, 103144, 103114]   
56          2015-12-22                             [103133, 103124, 103120]   
            2016-08-02                     [103144, 103154, 103119, 103149]   
            2017-04-20                             [103129, 103149, 103156]   
            2017-05-18                                             [103129]   
58          2016-04-07     [103103, 103109, 103105, 103148, 103147, 103161]   
            2017-09-18    [103129, 103105, 103116, 103124, 103139, 10310...   
61          2015-08-27                             [103133, 103142, 103149]   
            2016-07-08                             [103142, 103139, 103150]   
            2017-01-24                                             [103153]   
            2017-11-28                                     [103133, 103144]   
66          2016-03-22                                             [103149]   
            2017-07-03                                     [103154, 103143]   
67          2015-05-20                             [103119, 103124, 103145]   
            2016-04-01                     [103154, 103144, 103161, 103109]   
            2016-10-19                                     [103154, 103124]   
            2017-08-04     [103150, 103120, 103142, 103114, 103147, 103139]   
            2018-02-20                     [103131, 103154, 103120, 103149]   
73          2015-07-17             [103161, 103138, 103145, 103139, 103103]   
...                                                                     ...   
93826       2018-02-06             [103149, 103144, 103103, 103157, 103119]   
93835       2018-02-01                     [103103, 103102, 103149, 103154]   
93856       2018-03-06                     [103116, 103145, 103143, 103154]   
93883       2018-02-09                                             [103154]   
93911       2018-02-06                                     [103139, 103120]   
93912       2018-01-05                                     [103119, 103154]   
93913       2018-02-22                             [103131, 103149, 103154]   
93947       2018-02-13                                             [103116]   
93968       2017-11-20                                             [103142]   
93969       2017-12-21                                             [103148]   
93977       2017-12-19                                             [103127]   
94012       2018-01-12                             [103134, 103119, 103149]   
94040       2018-01-30                     [103149, 103133, 103139, 103116]   
94115       2018-01-30                                     [103149, 103119]   
94175       2018-02-02                                             [103149]   
94182       2018-02-12                     [103141, 103115, 103149, 103129]   
94189       2017-11-30                                             [103129]   
94190       2018-02-27                                             [103124]   
94231       2017-12-14      

In [212]:
df_grouped.index.names

FrozenList(['business_id', 'inspect_date'])

In [213]:
df_grouped.loc[(315,'2015-05-18'),:]

,,short_violation_id,risk_category
business_id,inspect_date,,
315,2015-05-18,"[103114, 103154, 103138]","[High Risk, Low Risk]"


# Use datetime type for inpsection dates with full set of data

In [214]:
df['short_violation_id'] = df['short_violation_id'].apply(np.int64)

In [215]:
# create a pandas series with groupby
m1100 = df.groupby(['business_id', 'inspect_date'])['short_violation_id'].unique()

In [216]:
# change series to dataframe
df_grouped2 = pd.DataFrame(m1100)

In [217]:
df_grouped2.loc[(315,'2015-05-18'),:]

,,short_violation_id
business_id,inspect_date,
315,2015-05-18,"[103114, 103154, 103138]"


## Create a frame for data filling in

In [218]:
ndf2 = df.drop_duplicates('business_id')

In [219]:
# idf2 is just a frame to be filled with data from df_grouped 
idf2 = ndf2.reset_index(drop=True)

In [220]:
idf2.at[:, 'p1_3'] = 0
idf2.at[:, 'p4_6'] = 0
idf2.at[:, 'p7_9'] = 0
idf2.at[:, 'p10_12'] = 0
idf2.at[:, 'p13_18'] = 0
idf2.at[:, 'p19_24'] = 0
idf2.at[:, 'p25_36'] = 0

### Filling up the number of violations per business per inspection date

In [221]:
for row in df_grouped2.iterrows():
    latest_enddate = pd.to_datetime('2018-03-31')
    bid = row[0][0]
    date = row[0][1]
    date1 = latest_enddate 
    date2 = latest_enddate - pd.DateOffset(months=3)
    date3 = latest_enddate - pd.DateOffset(months=6)
    date4 = latest_enddate - pd.DateOffset(months=9)
    date5 = latest_enddate - pd.DateOffset(months=12)
    date6 = latest_enddate - pd.DateOffset(months=18)
    date7 = latest_enddate - pd.DateOffset(months=24)
    date8 = latest_enddate - pd.DateOffset(months=36)

    if date2 < date <= date1:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p1_3'] = num_viols
    elif date3 < date <= date2:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p4_6'] = num_viols
    elif date4 < date <= date3:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p7_9'] = num_viols
    elif date5 < date <= date4:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p10_12'] = num_viols
    elif date6 < date <= date5:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p13_18'] = num_viols
    elif date7 < date <= date6:
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p19_24'] = num_viols
    elif date8 < date <= date7: 
        num_viols = len(row[1]['short_violation_id'])
        idx = idf2.business_id[idf2.business_id ==bid].index.tolist()
        idf2.at[idx, 'p25_36'] = num_viols

In [222]:
idf2.head(5)

,Unnamed: 0,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",...,20170928,103131,2017-09-28,0,0,5,0,0,6,5
1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",...,20161206,103157,2016-12-06,0,6,0,0,5,5,3
2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,...,20160503,103120,2016-05-03,4,0,2,0,3,3,2
3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,...,20160329,103113,2016-03-29,0,11,0,0,0,6,6


In [223]:
idf2.to_csv('data/ready4model_v2.csv')